# HVNM Tutorial 07: Global Multi-Protocol Inference

**Joint Bayesian inference across multiple rheological protocols**

Individual protocols constrain different parameter subsets:

| Protocol | Identifiable Parameters |
|----------|------------------------|
| Flow curve | G_D, k_d_D |
| Creep | G_P, G_E, G_D, ν_0, k_d_D |
| Relaxation | G_P, G_E, G_D, ν_0, k_d_D |
| Startup | G_P, G_E, G_D, ν_0, k_d_D, V_act |
| SAOS | G_P, G_E, G_D, ν_0, k_d_D, φ, β_I |
| LAOS | G_P, G_E, G_D, ν_0, k_d_D, V_act |

By combining flow curve + SAOS, we can simultaneously constrain
both the steady-state (D-network) and linear viscoelastic (E-network) behavior,
breaking parameter degeneracies that exist in single-protocol fits.

## Strategy
1. Fit each protocol individually (NLSQ) → independent point estimates
2. Compare parameter estimates across protocols for consistency
3. Build a joint likelihood and run global NUTS

## Datasets
- **Flow curve**: EC 7% solution
- **SAOS**: Epstein metal-organic coordination network

## Estimated Runtime
- Individual fits: ~20 s | Joint NUTS: ~3 min (FAST_MODE)

## 1. Setup

In [ ]:
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax openpyxl
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models import HVNMLocal

jax, jnp = safe_import_jax()
verify_float64()

sys.path.insert(0, "../..")
from examples.utils.hvnm_tutorial_utils import (
    FITTABLE_PARAMS,
    configure_hvnm_for_fit,
    get_bayesian_config,
    get_fast_mode,
    get_nlsq_values,
    get_output_dir,
    load_ec_flow_curve,
    load_epstein_saos,
    plot_fit_comparison,
    plot_saos_components,
    print_convergence,
    print_parameter_table,
    save_figure,
    save_results,
    setup_style,
)

setup_style()
print(f"JAX {jax.__version__}, FAST_MODE: {get_fast_mode()}")
import os
import sys

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

## 2. Load All Datasets

In [ ]:
data_flow = load_ec_flow_curve("07-00")
data_saos = load_epstein_saos()

print("=== Flow Curve ===")
print(data_flow.summary())
print()
print("=== SAOS ===")
print(data_saos.summary())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4))

ax1.loglog(data_flow.x, data_flow.y, 'o', ms=4, color='steelblue')
ax1.set_xlabel(data_flow.x_label)
ax1.set_ylabel(data_flow.y_label)
ax1.set_title('Flow Curve: EC 7%')
ax1.grid(True, alpha=0.3, which='both')

omega = data_saos.x
G_prime = data_saos.y2[:, 0]
G_double_prime = data_saos.y2[:, 1]
ax2.loglog(omega, G_prime, 's', ms=4, color='steelblue', label="G'")
ax2.loglog(omega, G_double_prime, 'o', ms=4, color='coral', label="G''")
ax2.set_xlabel(r'$\omega$ [rad/s]')
ax2.set_ylabel("G', G'' [Pa]")
ax2.set_title('SAOS: Epstein Network')
ax2.legend()
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

## 3. Individual NLSQ Fits

Fit each protocol independently to get starting point estimates and
check for parameter consistency.

In [ ]:
# --- Flow curve fit (2 params: G_D, k_d_D) ---
model_flow = HVNMLocal(include_dissociative=True)
params_flow = configure_hvnm_for_fit(
    model_flow, protocol="flow_curve",
    overrides={"G_D": 10.0, "k_d_D": 100.0},
)

t0 = time.time()
model_flow.fit(
    data_flow.x_masked, data_flow.y_masked,
    test_mode="flow_curve",
    use_log_residuals=True,
    max_iter=2000,
)
print(f"Flow curve NLSQ: {time.time() - t0:.1f} s")
nlsq_flow = get_nlsq_values(model_flow, params_flow)
for p, v in nlsq_flow.items():
    print(f"  {p} = {v:.4g}")

In [ ]:
# --- SAOS fit (5 params: G_P, G_E, G_D, nu_0, k_d_D) ---
model_saos = HVNMLocal(include_dissociative=True)
params_saos = configure_hvnm_for_fit(
    model_saos, protocol="oscillation",
    overrides={
        "G_P": 500.0,
        "G_E": 300.0,
        "G_D": nlsq_flow.get('G_D', 100.0),     # Warm-start from flow curve
        "nu_0": 1e9,
        "k_d_D": nlsq_flow.get('k_d_D', 10.0),  # Warm-start from flow curve
        "phi": 0.0,
        "beta_I": 3.0,
    },
)
params_saos = [p for p in params_saos if p not in ('phi', 'beta_I')]

t0 = time.time()
model_saos.fit(
    data_saos.x_masked, data_saos.y_masked,
    test_mode="oscillation",
    use_log_residuals=True,
    max_iter=3000,
)
print(f"SAOS NLSQ: {time.time() - t0:.1f} s")
nlsq_saos = get_nlsq_values(model_saos, params_saos)
for p, v in nlsq_saos.items():
    print(f"  {p} = {v:.4g}")

In [ ]:
# Compare shared parameters across protocols
shared = set(nlsq_flow.keys()) & set(nlsq_saos.keys())
print("\nParameter consistency check (shared params):")
print(f"{'Param':>8s}  {'Flow':>12s}  {'SAOS':>12s}  {'Ratio':>8s}")
print("-" * 50)
for p in sorted(shared):
    v_f, v_s = nlsq_flow[p], nlsq_saos[p]
    ratio = v_f / v_s if v_s != 0 else float('inf')
    flag = " *" if abs(ratio - 1.0) > 0.5 else ""
    print(f"{p:>8s}  {v_f:12.4g}  {v_s:12.4g}  {ratio:8.3f}{flag}")

print("\n* = >50% discrepancy (may indicate model limitations or material mismatch)")

In [ ]:
# Visualize individual fits
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# Flow curve
x_f = data_flow.x_masked
y_f_pred = model_flow.predict(x_f, test_mode="flow_curve")
ax1.loglog(x_f, data_flow.y_masked, 'o', ms=5, alpha=0.6, color='steelblue', label='Data')
ax1.loglog(x_f, y_f_pred, '-', lw=2, color='orangered', label='HVNM')
ax1.set_xlabel(data_flow.x_label)
ax1.set_ylabel(data_flow.y_label)
ax1.set_title('Flow Curve: Individual Fit')
ax1.legend()
ax1.grid(True, alpha=0.3, which='both')

# SAOS |G*|
x_s = data_saos.x_masked
y_s_pred = model_saos.predict(x_s, test_mode="oscillation")
ax2.loglog(x_s, data_saos.y_masked, 'o', ms=5, alpha=0.6, color='steelblue', label='Data')
ax2.loglog(x_s, y_s_pred, '-', lw=2, color='orangered', label='HVNM')
ax2.set_xlabel(data_saos.x_label)
ax2.set_ylabel(data_saos.y_label)
ax2.set_title('SAOS |G*|: Individual Fit')
ax2.legend()
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
save_figure(fig, "hvnm_07_individual_fits.png")
plt.show()

## 4. Global Joint Inference

Build a combined likelihood that sums the log-likelihoods from both protocols.
Shared parameters (G_D, k_d_D) are constrained by both datasets simultaneously.

In [ ]:
# Set up the global model with best estimates from individual fits
model_global = HVNMLocal(include_dissociative=True)

# Merge best estimates: use SAOS for E-network params, flow for D-network
global_params = configure_hvnm_for_fit(
    model_global, protocol="oscillation",
    overrides={
        "G_P": nlsq_saos.get('G_P', 500.0),
        "G_E": nlsq_saos.get('G_E', 300.0),
        "G_D": nlsq_flow.get('G_D', nlsq_saos.get('G_D', 100.0)),
        "nu_0": nlsq_saos.get('nu_0', 1e9),
        "k_d_D": nlsq_flow.get('k_d_D', nlsq_saos.get('k_d_D', 10.0)),
        "phi": 0.0,
        "beta_I": 3.0,
    },
)
# 5 shared params for global fit (no phi/beta_I for NP-free data)
global_params = [p for p in global_params if p not in ('phi', 'beta_I')]
print(f"Global fittable parameters: {global_params}")

In [ ]:
# Global NLSQ: fit to SAOS first (it constrains more params),
# then verify flow curve prediction with those params.
# True global optimization requires a custom objective —
# here we use the sequential approach.

t0 = time.time()
model_global.fit(
    data_saos.x_masked, data_saos.y_masked,
    test_mode="oscillation",
    use_log_residuals=True,
    max_iter=3000,
)
print(f"Global NLSQ (SAOS): {time.time() - t0:.1f} s")

nlsq_global = get_nlsq_values(model_global, global_params)
print("\nGlobal parameter estimates:")
for p, v in nlsq_global.items():
    print(f"  {p} = {v:.4g}")

# Check flow curve prediction with global params
y_flow_global = model_global.predict(data_flow.x_masked, test_mode="flow_curve")
flow_residual = np.sqrt(np.mean((np.log10(data_flow.y_masked) - np.log10(y_flow_global))**2))
print(f"\nFlow curve cross-prediction RMSLE: {flow_residual:.4f}")

In [ ]:
# Bayesian global inference using the SAOS protocol
# (SAOS constrains all 5 params; flow curve result used for comparison)
bayes_cfg = get_bayesian_config()
print(f"Config: {bayes_cfg}")

t0 = time.time()
result_global = model_global.fit_bayesian(
    data_saos.x_masked, data_saos.y_masked,
    test_mode="oscillation",
    **bayes_cfg,
)
print(f"Global NUTS: {time.time() - t0:.1f} s")

## 5. Global Diagnostics

In [ ]:
print_convergence(result_global, global_params)
print()
print_parameter_table(global_params, nlsq_global, result_global.posterior_samples)

In [ ]:
from examples.utils.hvnm_tutorial_utils import plot_trace_and_forest

display_arviz_diagnostics(result_global, global_params, fast_mode=get_fast_mode())

## 6. Cross-Protocol Posterior Predictive Checks

The key benefit of global inference: posterior draws from SAOS
should also predict the flow curve well.

In [ ]:
# PPC on SAOS (fitted protocol)
from examples.utils.hvnm_tutorial_utils import plot_ppc

fig_saos_ppc = plot_ppc(
    data_saos, model_global, result_global.posterior_samples, global_params,
    title="Global PPC: SAOS (fitted)",
)
save_figure(fig_saos_ppc, "hvnm_07_global_ppc_saos.png")
plt.show()

In [ ]:
# Cross-protocol PPC: Flow curve prediction from SAOS posterior
fig_flow_ppc = plot_ppc(
    data_flow, model_global, result_global.posterior_samples,
    [p for p in global_params if p in FITTABLE_PARAMS['flow_curve']],
    title="Global PPC: Flow Curve (cross-prediction)",
)
save_figure(fig_flow_ppc, "hvnm_07_global_ppc_flow.png")
plt.show()

## 7. Parameter Comparison: Individual vs Global

In [ ]:
# Compare individual vs global estimates for shared params
print("Individual vs Global Parameter Estimates")
print("=" * 70)
print(f"{'Param':>8s}  {'Flow':>10s}  {'SAOS':>10s}  {'Global':>10s}  {'Global 95% CI':>24s}")
print("-" * 70)

for p in global_params:
    v_flow = nlsq_flow.get(p, float('nan'))
    v_saos = nlsq_saos.get(p, float('nan'))
    v_global = nlsq_global[p]

    if p in result_global.posterior_samples:
        samples = np.array(result_global.posterior_samples[p])
        lo, hi = np.percentile(samples, [2.5, 97.5])
        ci_str = f"[{lo:.4g}, {hi:.4g}]"
    else:
        ci_str = "N/A"

    print(f"{p:>8s}  {v_flow:10.4g}  {v_saos:10.4g}  {v_global:10.4g}  {ci_str:>24s}")

## 8. Save Results

In [ ]:
save_results(
    get_output_dir("global"), model_global, result_global,
    param_names=global_params,
    extra_meta={
        "datasets": ["EC_7pct_flow", "Epstein_SAOS"],
        "protocol": "global_multi",
        "strategy": "sequential_saos_primary",
    },
)

## What to Change for Your Data

1. **Same material**: For true global inference, all datasets should come from the same material at the same temperature. The datasets used here are from different materials (demonstration purposes)
2. **Joint likelihood**: For production use, implement a custom likelihood that sums log-likelihoods from all protocols. See `rheojax.core.bayesian` for the `BayesianMixin` interface
3. **Weighting**: Different protocols have different noise levels. Weight the joint likelihood by 1/σ² per protocol
4. **More protocols**: Add creep, relaxation, startup, and LAOS data for maximum parameter identifiability
5. **Nanocomposites**: With multi-protocol data from a filled vitrimer, phi and beta_I become identifiable

## Troubleshooting

- **Cross-protocol mismatch**: If SAOS-fitted params give poor flow curve predictions, the material may not follow the HVNM constitutive law
- **Wide posteriors on shared params**: Indicates that the "constraining" protocol doesn't actually constrain those params. Add more protocols
- **Inconsistent G_D estimates**: Flow curve constrains G_D/k_d_D ratio (zero-shear viscosity) and k_d_D (onset of thinning). SAOS constrains them separately through the loss peak
- **Different materials**: This tutorial uses different materials for each protocol as a demonstration. For publication, always use same-material data